In [1]:
# Importing modules
import numpy as np
import pandas as pd
import numba
import pyarrow.dataset as ds

In [2]:
# Pandas display options (personal preference)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [3]:
# Importing the G09 parquet file
dataset = ds.dataset("../../data/WD10/WD10_2p4.parquet", format="parquet")
table = dataset.to_table()

# Setting uberID as index
df = table.to_pandas().set_index('uberID')
df.index=df.index.astype('int64')

del dataset,table

In [4]:
# Getting rid of weird spaces in the class column
df=df.replace({'class': {'ambiguous ': 'ambiguous',
                         'star      ': 'star', 
                         'galaxy    ': 'galaxy', 
                         'artefact  ':'artefact'}})

In [5]:
# Making a number of cuts: magnitude, mask, starmask and removing artefacts
df=df[(df['mag_Zt']<22) & 
      (df['mask']==0) & 
      (df['starmask']==0) & 
      (df['class']!='artefact')&
      (df['duplicate']==0)
]

In [6]:
# Getting a 1/10 subset of the data (for the purpose of this tutorial)
# df=df[::10]

In [7]:
df

,FrameName,FrameID,segID,xmax,ymax,censep,RAcen,Deccen,RAmax,Decmax,RAGAIA_r,RAGAIA,DecGAIA,DecGAIA_r,RAGAIA_r_cen,DecGAIA_r_cen,RAcen_gt,Deccen_gt,RAmax_gt,Decmax_gt,RAcen_rt,Deccen_rt,RAmax_rt,Decmax_rt,RAcen_Zt,Deccen_Zt,RAmax_Zt,Decmax_Zt,sky_mean,skyRMS_mean,log10seeing,log10seeing_r,log10seeing_i,log10seeing_Z,log10seeing_Y,mag,EBV,R50,R50_gt,R50_rt,R50_Yt,R50_Jt,R50_Ht,R90,R100,N100,axrat,ang,groupID,Ngroup,mag_app_Zt,mag_Zt,mag_app_gt,mag_app_rt,mag_app_it,mag_app_Yt,flux_ut,flux_err_ut,flux_ut_uncorrected,flux_err_ut_uncorrected,flux_gt,flux_err_gt,flux_gt_uncorrected,flux_err_gt_uncorrected,flux_rt,flux_err_rt,flux_rt_uncorrected,flux_err_rt_uncorrected,flux_it,flux_err_it,flux_it_uncorrected,flux_err_it_uncorrected,flux_Zt,flux_err_Zt,flux_Zt_uncorrected,flux_err_Zt_uncorrected,flux_Yt,flux_err_Yt,flux_Yt_uncorrected,flux_err_Yt_uncorrected,flux_Jt,flux_err_Jt,flux_Jt_uncorrected,flux_err_Jt_uncorrected,flux_Ht,flux_err_Ht,flux_Ht_uncorrected,flux_err_Ht_uncorrected,flux_Kt,flux_err_Kt,flux_Kt_uncorrected,flux_err_Kt_uncorrected,flux_W1t,flux_err_W1t,flux_W1t_uncorrected,flux_err_W1t_uncorrected,flux_W2t,flux_err_W2t,flux_W2t_uncorrected,flux_err_W2t_uncorrected,flux_uc,flux_err_uc,flux_gc,flux_err_gc,flux_rc,flux_err_rc,flux_ic,flux_err_ic,flux_Zc,flux_err_Zc,flux_Yc,flux_err_Yc,flux_Jc,flux_err_Jc,flux_Hc,flux_err_Hc,flux_Kc,flux_err_Kc,flux_W1c,flux_err_W1c,flux_W2c,flux_err_W2c,mask,starmask,starscol,starssize,class,noOPT_r,noOPT_i,noIR_Z,noIR_Y,duplicate,mag_Zc,mag_Zt_fake
uberID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
152041317902596,waves_postprocessed_151.5_3.2.rds,15204.0,402,13178.5,2595.5,7856.784139,150.984520,2.832859,150.984519,2.832849,150.984470,150.984470,2.832844,2.832844,150.984471,2.832840,150.984514,2.832847,150.984519,2.832849,150.984521,2.832844,150.984519,2.832849,150.984520,2.832859,150.984519,2.832849,0.005450,0.028003,-0.106948,-0.253420,-0.114577,-0.107245,-0.151243,16.076766,0.036432,0.775818,0.694059,0.550551,0.682726,0.675501,0.722055,2.088669,5.859128,1161,0.968855,121.396815,402,2,16.829423,15.994472,17.136094,16.526462,16.653912,16.572638,3.542453e-04,1.126320e-06,3.014309e-04,9.583967e-07,9.300822e-04,5.166496e-07,8.224620e-04,4.568679e-07,1.322346e-03,4.227617e-07,1.209650e-03,3.867320e-07,1.456031e-03,2.618840e-06,1.358113e-03,2.442722e-06,0.001453,1.216796e-06,0.001379,1.155040e-06,1.566832e-03,3.115313e-06,1.504342e-03,2.991065e-06,0.001285,3.938679e-06,0.001248,3.824557e-06,0.001119,0.000006,0.001098,0.000006,0.000806,0.000007,0.000796,0.000007,0.000320,2.931225e-06,0.000318,2.911498e-06,0.000136,2.272722e-06,0.000135,2.262088e-06,3.542453e-04,1.126320e-06,9.206818e-04,3.927114e-07,1.311417e-03,3.197012e-07,1.436595e-03,1.969294e-06,1.452818e-03,1.216796e-06,1.566832e-03,3.115313e-06,1.285407e-03,3.938679e-06,1.118659e-03,6.071062e-06,8.063744e-04,6.855420e-06,2.635329e-04,5.748836e-07,1.172000e-04,8.634734e-07,0.0,0.0,3.0,3.0,star,0.0,0.0,0.0,0.0,0.0,15.994472,0
152041399101268,waves_postprocessed_151.5_3.2.rds,15204.0,538,13990.5,1267.5,9222.415356,150.916831,2.722123,150.916841,2.722162,150.916791,150.916791,2.722157,2.722074,150.916779,2.722085,150.916830,2.722089,150.916841,2.722079,150.916829,2.722090,150.916841,2.722079,150.916833,2.722123,150.916841,2.722162,0.001188,0.029055,-0.106948,-0.253420,-0.114577,-0.107245,-0.151243,16.056690,0.030347,0.797116,0.707274,0.528856,0.709049,0.704649,0.672351,2.076544,5.894152,1132,0.933461,6.424889,538,1,16.800223,15.982716,18.242313,16.938558,16.836962,16.478748,2.779284e-05,8.978765e-07,2.429562e-05,7.848950e-07,3.232838e-04,4.972581e-07,2.918089e-04,4.488451e-07,8.729399e-04,4.160214e-07,8.105138e-04,3.862706e-07,1.194973e-03,1.965697e-06,1.127647e-03,1.854948e-06,0.001469,1.192177e-06,0.001406,1.141559e-06,1.746021e-03,2.645887e-06,1.687819e-03,2.557690e-06,0.001669,3.922070e-06,0.001628,3.827178e-06,0.001891,0.000007,0.001861,0.00000

In [10]:
# Reading the u and no u labels from notebooks 1 and 2, and creating new dataframes
df_u=pd.read_csv('../../data/WD10/WD10_u_no_bad_umap1.csv').set_index('uberID')
df_no_u=pd.read_csv('../../data/WD10/WD10_no_u_no_bad_umap1.csv').set_index('uberID')

In [11]:
# Creating new columns in the main dataframe with u and no u labels
df['cluster_label']=np.zeros(len(df))
df['cluster_label_no_u']=np.zeros(len(df))

df['label_type']=np.zeros(len(df))
df['label_type']='missing_band'

In [12]:
# Creating umap columns in the main dataframe with u and no u labels
for i in range(10):
    df[f'umap{i+1}_no_u'] = np.zeros(len(df))
    df[f'umap{i+1}_u'] = np.zeros(len(df))

In [13]:
# Filling the new columns with u and no u labels
df.loc[df_u.index,'cluster_label']=df_u['cluster_label'].apply(str)
df.loc[df_no_u.index,'cluster_label_no_u']=df_no_u['cluster_label'].apply(str)

/tmp/ipykernel_286722/2692133000.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['star' 'star' 'star' ... 'galaxy' 'star' 'star']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df_u.index,'cluster_label']=df_u['cluster_label'].apply(str)
/tmp/ipykernel_286722/2692133000.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['star' 'star' 'star' ... 'galaxy' 'star' 'star']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df_no_u.index,'cluster_label_no_u']=df_no_u['cluster_label'].apply(str)


In [14]:
for i in range(10):
    df.loc[df_u.index,f'umap{i+1}_u']=df_u[f'umap{i+1}']
    df.loc[df_no_u.index,f'umap{i+1}_no_u']=df_no_u[f'umap{i+1}']

In [15]:
labels=['star','galaxy']

In [16]:
# Creating a 'final label' column full of 1's to combine the two labels
df['final_label']=np.ones(len(df))


In [17]:
# Creating umap columns with 1's
for i in range(10):
    df[f'umap{i+1}'] = np.ones(len(df))


In [18]:
# First applying the no u labels to the final label
for label in labels:
    df.loc[df[df['cluster_label_no_u']==label].index,'final_label']=label
    df.loc[df[df['cluster_label_no_u']==label].index,'label_type']='no_u'

    # First putting umap columns with no u labels
    for i in range(10):
        df.loc[df[df['cluster_label_no_u']==label].index, f'umap{i+1}'] = df.loc[df[df['cluster_label_no_u']==label].index, f'umap{i+1}_no_u']

/tmp/ipykernel_286722/2715295979.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'star' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df[df['cluster_label_no_u']==label].index,'final_label']=label


In [19]:
# Then applying the u labels to the final label, overwriting any existing no u labels
for label in labels:
    df.loc[df[df['cluster_label']==label].index,'final_label']=label
    df.loc[df[df['cluster_label']==label].index,'label_type']='u'

    for i in range(10):
        df.loc[df[df['cluster_label']==label].index, f'umap{i+1}'] = df.loc[df[df['cluster_label']==label].index, f'umap{i+1}_u']

In [20]:
# Convering final label to string
df['final_label']=df['final_label'].apply(str)

In [21]:
# Creating a new column to indicate whether a source has any missing bands.
# If final label still == 1, that indicates a label hasn't been added, so it has a missing band
df['missing_bands']=True
df.loc[df[df['final_label']!='1.0'].index,'missing_bands']=False

In [22]:
# For the sources with missing labels we then overwrite with class
df.loc[df[df['final_label']=='1.0'].index,'final_label'] = df[df['final_label']=='1.0']['class']

In [23]:
# For sources with missing labels we put nan in the umap columns
for i in range(10):
    df.loc[df[df['missing_bands']==True].index, f'umap{i+1}'] = np.nan

In [24]:
# Sources with missing bands are only 1.5% of the sample
len(df[df['missing_bands']==True])/len(df)

0.04920221277373931

In [25]:
df

,FrameName,FrameID,segID,xmax,ymax,censep,RAcen,Deccen,RAmax,Decmax,RAGAIA_r,RAGAIA,DecGAIA,DecGAIA_r,RAGAIA_r_cen,DecGAIA_r_cen,RAcen_gt,Deccen_gt,RAmax_gt,Decmax_gt,RAcen_rt,Deccen_rt,RAmax_rt,Decmax_rt,RAcen_Zt,Deccen_Zt,RAmax_Zt,Decmax_Zt,sky_mean,skyRMS_mean,log10seeing,log10seeing_r,log10seeing_i,log10seeing_Z,log10seeing_Y,mag,EBV,R50,R50_gt,R50_rt,R50_Yt,R50_Jt,R50_Ht,R90,R100,N100,axrat,ang,groupID,Ngroup,mag_app_Zt,mag_Zt,mag_app_gt,mag_app_rt,mag_app_it,mag_app_Yt,flux_ut,flux_err_ut,flux_ut_uncorrected,flux_err_ut_uncorrected,flux_gt,flux_err_gt,flux_gt_uncorrected,flux_err_gt_uncorrected,flux_rt,flux_err_rt,flux_rt_uncorrected,flux_err_rt_uncorrected,flux_it,flux_err_it,flux_it_uncorrected,flux_err_it_uncorrected,flux_Zt,flux_err_Zt,flux_Zt_uncorrected,flux_err_Zt_uncorrected,flux_Yt,flux_err_Yt,flux_Yt_uncorrected,flux_err_Yt_uncorrected,flux_Jt,flux_err_Jt,flux_Jt_uncorrected,flux_err_Jt_uncorrected,flux_Ht,flux_err_Ht,flux_Ht_uncorrected,flux_err_Ht_uncorrected,flux_Kt,flux_err_Kt,flux_Kt_uncorrected,flux_err_Kt_uncorrected,flux_W1t,flux_err_W1t,flux_W1t_uncorrected,flux_err_W1t_uncorrected,flux_W2t,flux_err_W2t,flux_W2t_uncorrected,flux_err_W2t_uncorrected,flux_uc,flux_err_uc,flux_gc,flux_err_gc,flux_rc,flux_err_rc,flux_ic,flux_err_ic,flux_Zc,flux_err_Zc,flux_Yc,flux_err_Yc,flux_Jc,flux_err_Jc,flux_Hc,flux_err_Hc,flux_Kc,flux_err_Kc,flux_W1c,flux_err_W1c,flux_W2c,flux_err_W2c,mask,starmask,starscol,starssize,class,noOPT_r,noOPT_i,noIR_Z,noIR_Y,duplicate,mag_Zc,mag_Zt_fake,cluster_label,cluster_label_no_u,label_type,umap1_no_u,umap1_u,umap2_no_u,umap2_u,umap3_no_u,umap3_u,umap4_no_u,umap4_u,umap5_no_u,umap5_u,umap6_no_u,umap6_u,umap7_no_u,umap7_u,umap8_no_u,umap8_u,umap9_no_u,umap9_u,umap10_no_u,umap10_u,final_label,umap1,umap2,umap3,umap4,umap5,umap6,umap7,umap8,umap9,umap10,missing_bands
uberID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
152041317902596,waves_postprocessed_151.5_3.2.rds,15204.0,402,13178.5,2595.5,7856.784139,150.984520,2.832859,150.984519,2.832849,150.984470,150.984470,2.832844,2.832844,150.984471,2.832840,150.984514,2.832847,150.984519,2.832849,150.984521,2.832844,150.984519,2.832849,150.984520,2.832859,150.984519,2.832849,0.005450,0.028003,-0.106948,-0.253420,-0.114577,-0.107245,-0.151243,16.076766,0.036432,0.775818,0.694059,0.550551,0.682726,0.675501,0.722055,2.088669,5.859128,1161,0.968855,121.396815,402,2,16.829423,15.994472,17.136094,16.526462,16.653912,16.572638,3.542453e-04,1.126320e-06,3.014309e-04,9.583967e-07,9.300822e-04,5.166496e-07,8.224620e-04,4.568679e-07,1.322346e-03,4.227617e-07,1.209650e-03,3.867320e-07,1.456031e-03,2.618840e-06,1.358113e-03,2.442722e-06,0.001453,1.216796e-06,0.001379,1.155040e-06,1.566832e-03,3.115313e-06,1.504342e-03,2.991065e-06,0.001285,3.938679e-06,0.001248,3.824557e-06,0.001119,0.000006,0.001098,0.000006,0.000806,0.000007,0.000796,0.000007,0.000320,2.931225e-06,0.000318,2.911498e-06,0.000136,2.272722e-06,0.000135,2.262088e-06,3.542453e-04,1.126320e-06,9.206818e-04,3.927114e-07,1.311417e-03,3.197012e-07,1.436595e-03,1.969294e-06,1.452818e-03,1.216796e-06,1.566832e-03,3.115313e-06,1.285407e-03,3.938679e-06,1.118659e-03,6.071062e-06,8.063744e-04,6.855420e-06,2.635329e-04,5.748836e-07,1.172000e-04,8.634734e-07,0.0,0.0,3.0,3.0,star,0.0,0.0,0.0,0.0,0.0,15.994472,0,star,star,u,5.757523,4.328341,5.335961,4.584031,-1.735326,-1.595363,7.470030,6.220344,5.272532,6.182676,5.254004,5.242071,6.497044,7.637447,6.058050,6.619080,4.135395,3.819964,5.329246,4.461609,star,4.328341,4.584031,-1.595363,6.220344,6.182676,5.242071,7.637447,6.619080,3.819964,4.461609,False
152041399101268,waves_postprocessed_151.5_3.2.rds,15204.0,538,13990.5,1267.5,9222.415356,150.916831,2.722123,150.916841,2.722162,150.916791,150.916791,2.722157,2.722074,150.916779,2.722085,150.916830,2.722089,150.916841,2.722079,150.916829,2.722090,150.916841,2.722079,150.916833,2.722

In [26]:
df['uberID']=df.index

In [27]:
df = df.rename(columns={'cluster_label':'cluster_label_u'})

In [28]:
# Saving
df.to_csv('../../data/WD10/WD10_UMAP_full.csv')

In [29]:
columns_to_save = ['uberID', 'RAmax', 'Decmax', 'mag_Zt','final_label', 'missing_bands', 'umap1', 'umap2', 'umap3', 'umap4', 'umap5', 'umap6', 'umap7', 'umap8', 'umap9', 'umap10', 'class',
                   'cluster_label_u', 'cluster_label_no_u', 'label_type', 'umap1_u', 'umap2_u', 'umap3_u', 'umap4_u', 'umap5_u', 'umap6_u', 'umap7_u', 'umap8_u', 'umap9_u', 'umap10_u',
                   'umap1_no_u', 'umap2_no_u', 'umap3_no_u', 'umap4_no_u', 'umap5_no_u', 'umap6_no_u', 'umap7_no_u', 'umap8_no_u', 'umap9_no_u', 'umap10_no_u']

In [30]:
df[columns_to_save].to_csv('../../data/WD10/WD10_UMAP_reduced_columns.csv')